In [1]:
from bayesian_optimization import BayesianOptimization
import pandas as pd 
from botorch.test_functions import Ackley
from botorch.test_functions import Levy 

# Variables that determine the size of the dataset
N_INITIAL = 100
EPOCHS = 10
BATCH_SIZE = 10
DIM = 10
# Bounds are 10 for Levy and 32 for Ackley 

optimizer = BayesianOptimization(fun=Ackley(dim=DIM, negate=True), 
                                 batch_size=BATCH_SIZE, 
                                 dim=DIM, 
                                 epochs=EPOCHS, 
                                 n_init=N_INITIAL, 
                                 lower_bound=-32.176,
                                 upper_bound=32.176,
                                 seed=5,
                                 acqf_type='qUCB')

x_max, y_max = optimizer.run()
data = optimizer.get_data()
full = optimizer.format(data, dim=DIM, n_init=N_INITIAL, batch_size=BATCH_SIZE, epochs=EPOCHS)
display(full)

/Users/dylanschubert/anaconda3/envs/msc_project/lib/python3.12/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


,Batch,x1,x2,x3,x4,x5,x6,x7,x8,x9,x10,y
0,0.0,-1.268344,-24.630157,-24.197015,-9.895261,-18.438448,3.098315,-4.773726,27.769696,8.628926,-4.579019,-20.991023
1,0.0,11.050301,11.118967,16.558254,23.985074,0.761052,-20.643286,25.763249,-29.443123,-19.246696,26.092024,-21.243080
2,0.0,31.271793,-8.915932,-5.679147,-28.166922,19.969438,-0.032730,-30.915526,-0.165285,-14.733306,10.061175,-20.617730
3,0.0,-20.763086,27.831074,13.836867,2.068094,-1.302439,23.736404,3.641597,7.022896,21.208903,-23.626410,-20.943148
4,0.0,-25.446474,-6.089227,30.476108,12.502565,-14.907971,10.281443,-21.592235,-12.709945,-7.720115,18.108712,-21.307879
...,...,...,...,...,...,...,...,...,...,...,...,...
195,10.0,6.276859,-19.517810,9.165833,32.096925,2.817504,-5.817941,0.979476,-31.901072,12.284349,19.388695,-20.935490
196,10.0,-5.793735,-8.487261,20.861256,25.601760,-15.550696,9.987418,4.145270,-14.100150,11.849820,-11.055327,-20.314919
197,10.0,0.470168,-5.865362,3.699474,7.934607,0.391342,-6.489089,-2.900866,-8.615958,5.333767,-4.688156,-15.068747
198,10.0,2.417947,-13.697862,-5.660556,27.756570,8.384915,-13.542080,-31.584724,20.062906,18.282627,16.378456,-21.522593


In [5]:
import pandas as pd
cols = ['Batch','y']
df = full[cols]


y_0 = df[df['Batch'] == 0.0][['y']]
y_1 = df[df['Batch'] == 1.0][['y']]


def get_batches(id1,id2):
    y_0 = df[df['Batch'] == id1][['y']]
    y_1 = df[df['Batch'] == id2][['y']]
    return y_0, y_1


In [6]:
import numpy as np

def bootstrap_diff_max(X, Y, n_bootstrap=10000):
    np.random.seed(0)
    max_diffs = []
    for _ in range(n_bootstrap):
        X_resample = np.random.choice(X, size=len(X), replace=True)
        Y_resample = np.random.choice(Y, size=len(Y), replace=True)
        max_diff = np.max(X_resample) - np.max(Y_resample)
        max_diffs.append(max_diff)
    return np.array(max_diffs)

# Bootstrap method
max_diffs_bootstrap = bootstrap_diff_max(y_0, y_1)
obs_diff = np.max(y_0) - np.max(y_1)

# Confidence interval
lower_bound_bootstrap = np.percentile(max_diffs_bootstrap, 2.5)
upper_bound_bootstrap = np.percentile(max_diffs_bootstrap, 97.5)

print(f"Bootstrap Observed difference of maxima: {obs_diff}")
print(f"Bootstrap 95% Confidence interval: [{lower_bound_bootstrap}, {upper_bound_bootstrap}]")

ValueError: a must be 1-dimensional

In [3]:
def permutation_test_max(X, Y, n_permutations=10000):
    np.random.seed(0)
    combined = np.concatenate([X, Y])
    n_X = len(X)
    obs_diff = np.max(X) - np.max(Y)
    
    perm_diffs = []
    for _ in range(n_permutations):
        np.random.shuffle(combined)
        X_permuted = combined[:n_X]
        Y_permuted = combined[n_X:]
        max_diff = np.max(X_permuted) - np.max(Y_permuted)
        perm_diffs.append(max_diff)
        
    p_value = np.mean(np.abs(perm_diffs) >= np.abs(obs_diff))
    return np.array(perm_diffs), obs_diff, p_value

# Permutation test
perm_diffs, obs_diff_perm, p_value = permutation_test_max(X, Y)

print(f"Permutation Observed difference of maxima: {obs_diff_perm}")
print(f"Permutation p-value: {p_value}")


Permutation Observed difference of maxima: -0.5953076628331537
Permutation p-value: 1.0
